In [190]:
import random
import tree_sitter_python as tspython
from tree_sitter import Language, Parser

In [4]:
# Build parser with python language
PY_LANGUAGE = Language(tspython.language())
parser = Parser(PY_LANGUAGE)

In [417]:
# use dataset sample
import pandas as pd
data = pd.read_parquet("dataset.parquet")
code = bytes(data[data.split_name == "train"]["func_code_string"][0], "utf-8")

In [416]:
# simple sample, need to be bytes not string, if string is sent byte(string) could be used
code = b"""def hello_world(a,b):
    return a + b

# comment example def new_method(d):
def test(c):
    if true:
        return 0
    else:
        return c
def return_string():
    return "hello\""""

identifiers are basically variables and function names

In [418]:
tree = parser.parse(code,encoding="utf8")
help(tree)

Help on Tree in module tree_sitter object:

class Tree(builtins.object)
 |  A tree that represents the syntactic structure of a source code file.
 |
 |  Methods defined here:
 |
 |  __copy__(self, /)
 |      Use :func:`copy.copy` to create a copy of the tree.
 |
 |  changed_ranges(self, /, new_tree)
 |      Compare this old edited syntax tree to a new syntax tree representing the same document, returning a sequence of ranges whose syntactic structure has changed.
 |
 |      Returns
 |      -------
 |      Ranges where the hierarchical structure of syntax nodes (from root to leaf) has changed between the old and new trees. Characters outside these ranges have identical ancestor nodes in both trees.
 |
 |      Note
 |      ----
 |      The returned ranges may be slightly larger than the exact changed areas, but Tree-sitter attempts to make them as small as possible.
 |
 |      Tip
 |      ---
 |      For this to work correctly, this syntax tree must have been edited such that its ranges 

In [327]:
print(tree.language)
print(tree.included_ranges)
print(tree.root_node)


<Language id=140737184724032, version=15, name="python">
[<Range start_point=(0, 0), end_point=(4294967295, 4294967295), start_byte=0, end_byte=4294967295>]
(module (function_definition name: (identifier) parameters: (parameters (identifier) (identifier)) body: (block (return_statement (binary_operator left: (identifier) right: (identifier))))) (function_definition name: (identifier) parameters: (parameters (identifier)) body: (block (if_statement condition: (identifier) consequence: (block (return_statement (integer))) alternative: (else_clause body: (block (return_statement (identifier))))))) (function_definition name: (identifier) parameters: (parameters) body: (block (return_statement (string (string_start) (string_content) (string_end))))))


In [419]:
node = tree.root_node

In [329]:
help(node)

Help on Node in module tree_sitter object:

class Node(builtins.object)
 |  A single node within a syntax ``Tree``.
 |
 |  Methods defined here:
 |
 |  __eq__(self, value, /)
 |      Return self==value.
 |
 |  __ge__(self, value, /)
 |      Return self>=value.
 |
 |  __gt__(self, value, /)
 |      Return self>value.
 |
 |  __hash__(self, /)
 |      Return hash(self).
 |
 |  __le__(self, value, /)
 |      Return self<=value.
 |
 |  __lt__(self, value, /)
 |      Return self<value.
 |
 |  __ne__(self, value, /)
 |      Return self!=value.
 |
 |  __repr__(self, /)
 |      Return repr(self).
 |
 |  __str__(self, /)
 |      Return str(self).
 |
 |  child(self, index, /)
 |      Get this node's child at the given index, where ``0`` represents the first child.
 |
 |      Caution
 |      -------
 |      This method is fairly fast, but its cost is technically ``log(i)``, so if you might be iterating over a long list of children, you should use :attr:`children` or :meth:`walk` instead.
 |
 |  ch

In [387]:
# text is in utf-8 so we need to decode it for string
node.type, node.text.decode('utf8'), node.children# type: ignore

('module',
 'def hello_world(a,b):\n    return a + b\n\n# comment example def new_method(d):\ndef test(c):\n    if true:\n        return 0\n    else:\n        return c\ndef return_string():\n    return "hello"',
 [<Node type=function_definition, start_point=(0, 0), end_point=(1, 16)>,
  <Node type=comment, start_point=(3, 0), end_point=(3, 36)>,
  <Node type=function_definition, start_point=(4, 0), end_point=(8, 16)>,
  <Node type=function_definition, start_point=(9, 0), end_point=(10, 18)>])

In [ ]:
identifiers = []
def rec_tree(node):  
    for children_node in node.children:

        #print(f"type: {children_node.type}, text: {children_node.text}")
        if children_node.type == 'identifier': #and random.choice([True, False]):
            # point takes the row column position, byte is just the position of the byte
            # print(children_node.start_point, children_node.start_byte)
            # print(children_node.end_point, children_node.end_byte)
            identifiers.append((children_node.start_byte, children_node.end_byte))
            
        if children_node.type == 'comment':
            continue

        rec_tree(children_node)
        
    return identifiers

result = rec_tree(node)
print(len(result))

381


In [372]:
from torchtext.data import get_tokenizer

tokenizer = get_tokenizer("basic_english")


In [373]:
code.decode()

'def get_figure(self, **kwargs):\n        """Get a plotly figure of the heatmap."""\n\n        emin = kwargs.pop(\'emin\', -1.0)\n        emax = kwargs.pop(\'emax\', 1.0)\n        width = kwargs.pop(\'width\', 800)\n        height = kwargs.pop(\'height\', 600)\n        margin_left = kwargs.pop(\'margin_left\', 100)\n        margin_bottom = kwargs.pop(\'margin_bottom\', 60)\n        margin_top = kwargs.pop(\'margin_top\', 30)\n        margin_right = kwargs.pop(\'margin_right\', 0)\n        colorbar_size = kwargs.pop(\'colorbar_size\', 0.4)\n\n        xaxis_label = kwargs.pop(\'xaxis_label\', None)\n        yaxis_label = kwargs.pop(\'yaxis_label\', None)\n        xticks_angle = kwargs.pop(\'xaxis_angle\', 30)\n        font = kwargs.pop(\'font\', \'"Droid Serif", "Open Serif", serif\')\n        font_size = kwargs.pop(\'font_size\', 12)\n        title = kwargs.pop(\'title\', None)\n        title_font_size = kwargs.pop(\'title_font_size\', None)\n        annotation_font_size = kwargs.pop(\'

In [374]:
# mask 15% of the tokens
tokens_sample = tokenizer(code.decode())
num_tokens_to_mask = (len(tokens_sample)) * 15 // 100
num_tokens_to_mask

179

In [393]:
# in case we have fewer identifier tokens to mask  then the number of tokens
# just take call the possible identifiers
num_tokens_to_mask =  num_tokens_to_mask if len(result) > num_tokens_to_mask else len(result)
num_tokens_to_mask

10

In [394]:
to_mask = random.sample(result, num_tokens_to_mask)
# need to sort the values as to modify the string from left to right
to_mask = sorted(to_mask, key= lambda x: x[0])
to_mask

[(4, 15),
 (16, 17),
 (18, 19),
 (33, 34),
 (37, 38),
 (81, 85),
 (86, 87),
 (97, 101),
 (145, 146),
 (151, 164)]

In [ ]:
mod = node.text
difference = 0
offset = 0
print(node.text)
for mask in to_mask:
    start, end = mask[0], mask[1]

    label = mod[start + offset  :end + offset ]
    mod = mod[: start + offset] + b'<mask>' + mod[end + offset: ]
    
    
    # because we modify the string, we need an offest for the modifications
    difference = end - start # offset
    offset += 6 - difference

print(mod)
    

b'def get_figure(self, **kwargs):\n        """Get a plotly figure of the heatmap."""\n\n        emin = kwargs.pop(\'emin\', -1.0)\n        emax = kwargs.pop(\'emax\', 1.0)\n        width = kwargs.pop(\'width\', 800)\n        height = kwargs.pop(\'height\', 600)\n        margin_left = kwargs.pop(\'margin_left\', 100)\n        margin_bottom = kwargs.pop(\'margin_bottom\', 60)\n        margin_top = kwargs.pop(\'margin_top\', 30)\n        margin_right = kwargs.pop(\'margin_right\', 0)\n        colorbar_size = kwargs.pop(\'colorbar_size\', 0.4)\n\n        xaxis_label = kwargs.pop(\'xaxis_label\', None)\n        yaxis_label = kwargs.pop(\'yaxis_label\', None)\n        xticks_angle = kwargs.pop(\'xaxis_angle\', 30)\n        font = kwargs.pop(\'font\', \'"Droid Serif", "Open Serif", serif\')\n        font_size = kwargs.pop(\'font_size\', 12)\n        title = kwargs.pop(\'title\', None)\n        title_font_size = kwargs.pop(\'title_font_size\', None)\n        annotation_font_size = kwargs.pop(\